In [57]:
import pandas as pd

from nltk.stem.snowball import EnglishStemmer
from copy import deepcopy
snowball = EnglishStemmer()
from bs4 import BeautifulSoup
import unicodedata
from nltk.tokenize import sent_tokenize
import re
# from sklearn
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [58]:
data_ = pd.read_csv("sms_spam.csv")

In [42]:
def remove_whitespace_and_special_chars(text):
        """general text cleaning"""
        """Includes removal of extra whitespace, special characters"""
        
        tab_newline_pattern = '[\t\n]'
        multi_space = ' {2,}'
        special_chars = '[^a-zA-Z0-9 ]'        
        formatted_text = re.sub(tab_newline_pattern, ' ', text)
        formatted_text = re.sub(multi_space, ' ', formatted_text)
        formatted_text = re.sub(special_chars, ' ', formatted_text)
        formatted_text = re.sub(multi_space, ' ', formatted_text)
        return formatted_text
def removeStopWord(s):
        """ Removing English stopwords from the text """
        
        stop = set(stopwords.words('english'))
        return " ".join([words for words in word_tokenize(s) if words not in stop])
    
def remove_non_ascii(text):
        """Remove non-ASCII characters from list of tokenized words"""
        text = re.sub(r'\x85', '', text) # replace ellipses
        text = re.sub(r'\x91', '', text)  # replace left single quote
        text = re.sub(r'\x92', '', text)  # replace right single quote
        text = re.sub(u'\x93', '', text)  # replace left double quote
        text = re.sub(u'\x94', '', text)  # replace right double quote
        text = re.sub(r'\x95', '', text)   # replace bullet
        text = re.sub(r'\x96', '', text)   # replace bullet
        text = re.sub(r'\x99', '', text)   #replace TM
        text = re.sub(r'\xae', '', text)    # replace (R)
        text = re.sub(r'\xb0', '', text)    # replace degree symbol
        text = re.sub(r'\xba', '', text)    # replace degree symbol
        new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        return new_text    
def snowball_stemmer(token):
            return snowball.stem(token)
def stemmer(text):
        return " ".join([snowball_stemmer(w) for w in text.split()]).strip()
    
def transform(x):
        text = remove_non_ascii(x.lower())
        text = remove_whitespace_and_special_chars(text)
        text = removeStopWord(text)
        # text = stemmer(text)
        return text
    

In [43]:
data_['text'] = data_['text'].apply(transform)


In [ ]:
mapping_dict={"spam":0,"ham":1}
data_['type'] = 

In [52]:
data_.to_csv("traning_data_3.csv",index=False)

In [53]:
pd.read_csv("traning_data_3.csv")

,type,text
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah think goes usf lives around though
...,...,...
5569,spam,2nd time tried 2 contact u u 750 pound prize 2...
5570,ham,u b going esplanade fr home
5571,ham,pity mood suggestions
5572,ham,guy bitching acted like interested buying some...


In [59]:
import torch
import pandas as pd

# nlp library of Pytorch
from torchtext.legacy import data

import warnings as wrn
wrn.filterwarnings('ignore')
SEED = 2021

torch.manual_seed(SEED)
torch.backends.cuda.deterministic = True


In [60]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)
fields = [("type",LABEL),('text',TEXT)]


In [61]:
training_data = data.TabularDataset(path="sms_spam.csv",
                                    format="csv",
                                    fields=fields,
                                    skip_header=True
                                   )

print(vars(training_data.examples[0]))

{'type': 'ham', 'text': ['Go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'Cine', 'there', 'got', 'amore', 'wat', '...']}


In [62]:
import random
# train and validation splitting
train_data,valid_data = training_data.split(split_ratio=0.75,
                                            random_state=random.seed(SEED))

In [63]:
TEXT.build_vocab(train_data,
                 min_freq=5)

LABEL.build_vocab(train_data)

In [64]:
print("Size of text vocab:",len(TEXT.vocab))


Size of text vocab: 1751


In [65]:
print("Size of label vocab:",len(LABEL.vocab))


Size of label vocab: 2


In [78]:
device = torch.device("cuda")

BATCH_SIZE = 64

# We'll create iterators to get batches of data when we want to use them
"""
This BucketIterator batches the similar length of samples and reduces the need of 
padding tokens. This makes our future model more stable

"""
train_iterator,validation_iterator = data.BucketIterator.splits(
    (train_data,valid_data),
    batch_size = BATCH_SIZE,
    # Sort key is how to sort the samples
    sort_key = lambda x:len(x.text),
    sort_within_batch = True,
    device = device
)

In [79]:
import torch.nn as nn

class LSTMNet(nn.Module):
    
    def __init__(self,vocab_size,embedding_dim,hidden_dim,output_dim,n_layers,bidirectional,dropout):
        
        super(LSTMNet,self).__init__()
        
        # Embedding layer converts integer sequences to vector sequences
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        
        # LSTM layer process the vector sequences 
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers = n_layers,
                            bidirectional = bidirectional,
                            dropout = dropout,
                            batch_first = True
                           )
        
        # Dense layer to pr
        self.fc = nn.Linear(hidden_dim * 2,output_dim)
        # Prediction activation function
        self.sigmoid = nn.Sigmoid()
        
    
    def forward(self,text,text_lengths):
        embedded = self.embedding(text)
        
        # Thanks to packing, LSTM don't see padding tokens 
        # and this makes our model better
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(),batch_first=True)
        
        packed_output,(hidden_state,cell_state) = self.lstm(packed_embedded)
        
        # Concatenating the final forward and backward hidden states
        hidden = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)
        
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.sigmoid(dense_outputs)
        
        return outputs
    


In [80]:
SIZE_OF_VOCAB = len(TEXT.vocab)
EMBEDDING_DIM = 100
NUM_HIDDEN_NODES = 64
NUM_OUTPUT_NODES = 1
NUM_LAYERS = 2
BIDIRECTION = True
DROPOUT = 0.2

In [81]:
model = LSTMNet(SIZE_OF_VOCAB,
                EMBEDDING_DIM,
                NUM_HIDDEN_NODES,
                NUM_OUTPUT_NODES,
                NUM_LAYERS,
                BIDIRECTION,
                DROPOUT
               )

In [82]:
import torch.optim as optim
model = model.to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-4)
criterion = nn.BCELoss()
criterion = criterion.to(device)


In [83]:
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [84]:
def train(model,iterator,optimizer,criterion):
    
    epoch_loss = 0.0
    epoch_acc = 0.0
    
    model.train()
    
    for batch in iterator:
        
        # cleaning the cache of optimizer
        optimizer.zero_grad()
        
        text,text_lengths = batch.text
        
        # forward propagation and squeezing
        predictions = model(text,text_lengths).squeeze()
        
        # computing loss / backward propagation
        loss = criterion(predictions,batch.type)
        loss.backward()
        
        # accuracy
        acc = binary_accuracy(predictions,batch.type)
        
        # updating params
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    # It'll return the means of loss and accuracy
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [85]:
def evaluate(model,iterator,criterion):
    
    epoch_loss = 0.0
    epoch_acc = 0.0
    
    # deactivate the dropouts
    model.eval()
    
    # Sets require_grad flat False
    with torch.no_grad():
        for batch in iterator:
            text,text_lengths = batch.text
            
            predictions = model(text,text_lengths).squeeze()
              
            #compute loss and accuracy
            loss = criterion(predictions, batch.type)
            acc = binary_accuracy(predictions, batch.type)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [86]:
EPOCH_NUMBER = 15
for epoch in range(1,EPOCH_NUMBER+1):
    
    train_loss,train_acc = train(model,train_iterator,optimizer,criterion)
    
    valid_loss,valid_acc = evaluate(model,validation_iterator,criterion)
    
    # Showing statistics
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print()

	Train Loss: 0.651 | Train Acc: 73.44%
	 Val. Loss: 0.572 |  Val. Acc: 88.26%

	Train Loss: 0.502 | Train Acc: 86.22%
	 Val. Loss: 0.406 |  Val. Acc: 88.26%

	Train Loss: 0.406 | Train Acc: 86.22%
	 Val. Loss: 0.360 |  Val. Acc: 88.26%

	Train Loss: 0.363 | Train Acc: 86.22%
	 Val. Loss: 0.321 |  Val. Acc: 88.26%

	Train Loss: 0.316 | Train Acc: 86.22%
	 Val. Loss: 0.272 |  Val. Acc: 88.26%

	Train Loss: 0.268 | Train Acc: 86.62%
	 Val. Loss: 0.233 |  Val. Acc: 89.75%

	Train Loss: 0.232 | Train Acc: 88.66%
	 Val. Loss: 0.205 |  Val. Acc: 90.96%

	Train Loss: 0.208 | Train Acc: 90.91%
	 Val. Loss: 0.186 |  Val. Acc: 92.66%

	Train Loss: 0.188 | Train Acc: 92.21%
	 Val. Loss: 0.173 |  Val. Acc: 94.01%

	Train Loss: 0.169 | Train Acc: 94.15%
	 Val. Loss: 0.158 |  Val. Acc: 94.65%

	Train Loss: 0.150 | Train Acc: 95.60%
	 Val. Loss: 0.138 |  Val. Acc: 95.65%

	Train Loss: 0.130 | Train Acc: 96.45%
	 Val. Loss: 0.123 |  Val. Acc: 96.22%

	Train Loss: 0.110 | Train Acc: 97.02%
	 Val. Loss: 